# Agent Evaluation

Now that we have our custom DIY Agent with ML tools, we can evaluate how the model is calling our tools and evaluate traces in detail.

In [1]:
%pip install uv

/Users/scott.mckean/Repos/hydrate_prediction/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%sh uv pip install .

UsageError: Line magic function `%sh` not found (But cell magic `%%sh` exists, did you mean that instead?).


## Evaluation Dataset

You can edit the requests or expected responses in your evaluation dataset and run evaluation as you iterate your agent, leveraging mlflow to track the computed quality metrics.

Evaluate your agent with one of our [predefined LLM scorers](https://learn.microsoft.com/azure/databricks/mlflow3/genai/eval-monitor/predefined-judge-scorers), or try adding [custom metrics](https://learn.microsoft.com/azure/databricks/mlflow3/genai/eval-monitor/custom-scorers).


In [ ]:
import mlflow
from mlflow.genai.scorers import RelevanceToQuery, Safety, RetrievalRelevance, RetrievalGroundedness
from databricks import agents

eval_dataset = [
    {
        "inputs": {
            "messages": [
                {
                    "role": "user",
                    "content": "What are the predictions for well 15 for the next minute?"
                }
            ]
        },
        "expected_response": None
    }
]

eval_results = mlflow.genai.evaluate(
    data=eval_dataset,
    predict_fn=lambda messages: AGENT.predict({"messages": messages}),
    scorers=[RelevanceToQuery(), Safety()], # add more scorers here if they're applicable
)

# Review the evaluation results in the MLfLow UI (see console output)


## Trace Evaluation

One of the most powerful features of MLflow is the ability to trace the execution of a model. This allows us to see the exact inputs and outputs of a model, as well as the code that was used to train it. This is especially important for real tasks where an agent is calling an ML model.

## Guidelines and Custom Scorers
Correctness is great, but we often need much more granular information about how a model is doing.

## Feedback
The Review App is a great way to capture feedback, but with `mlflow-tracing`, we can now capture feedback directly from users into a table.